# Training

In this section you can train the LSTM model with videogame sounds. The first section is monophony and the second with polyphony encoding.

## Train monophony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping for visualisation
# mapping.save("mapping.txt")

In [ ]:
from pathlib import Path
# Load songs
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))
print(f"Found {len(paths)} songs.")

encoded_songs = utils.load_dataset_lstm(paths, 32, mapping, corpus=False, flat = False)

Each node is represented like this:
```
c  = n60
c# = n61
d  = n62
...
```

To represent the duration of each note, we will use "`_`" for 1/16. For example, if we want to represent a C note with a duration of 1/8, we will use "`n60 _ _`". If we want to represent a C# note with a duration of 1/16, we will use onyl the letter "`n61`".

To sepperate each song there are multiple "`/`" in the dataset.

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

# For tensorboard logging
logdir = "logs/videospielmusik_lstm_solo"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=50, batch_size=32, tensorboard_callback=tensorboard_callback)

To load the pre trainend model you have to save the model. It will be saved at `mukkeBude/model/preTrainedModels`

In [ ]:
model.save("Videospielmusik_soloMelodie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/videospielmusik_lstm_solo

## Train polyphony encoding

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import music21 as m21
import tensorflow as tf
import keras

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
from pathlib import Path
# Load songs
paths = list(Path("../mukkeBude/songs/pokemon").rglob("*.mid"))
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.extend(encoded_song)

print(f"Songs encoded: {len(encoded_songs)}")

Each node is represented like this:
```
c  = n60
c# = n61
d  = n62
...
```

To represent the duration of each note, we will use "`d1, d2, d3, ...`". For example, if we want to represent a C note with a duration of 1/8, we will use "`n60 d2`". If we want to represent a C# note with a duration of 1/16, we will use "`n61 d1`".

To sepperate the notes which are played at the same time we will use "`xxsep`". For example, if we want to represent a C note with a duration of 1/16 and a C# note with a duration of 1/16 played at the same time, we will use "`n60 d1 n61 d1 xxsep`".

To sepperate each song there is "`xxbos`" in the dataset.

In [ ]:
# Train model
model = MukkeBudeLSTM(mapping)
print(model)

logdir = "logs/videospielmusik_lstm_polyphonie"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.train(encoded_songs, epochs=50, batch_size=32, tensorboard_callback=tensorboard_callback)

In [ ]:
# Save model
model.save("Videospielmusik_polyphonie_lstm")

In [ ]:
# Show the logs
%tensorboard --logdir logs/videospielmusik_lstm_polyphonie

# Generate music

In this section you can generate music with a pre trained LSTM model. The first section is monophony and the second with polyphony encoding.

## Generate monophony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import REST
from mukkeBude.mapping import WAIT_LSTM


# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
model = MukkeBudeLSTM.load(mapping, "Videospielmusik_soloMelodie_lstm")

You need to pass a start seed. You can also give the network a max length for the generation (it can be shorter but not longer)

In [ ]:
# Create song
generated_song = model.generate("n79 _ n67 n67 n67 _ n74 n79 n77 _ n65 n65 n65 _ n72 n77 n76 _ n60 n60 n60 _ n76 n76 n76 _ _ n72 n67 n76 n78 _ _ _ n74 n69 n81 n67 _ _ _ _ _ _ _ _ _ _", max_length=800, temperature=0.7)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
# They should not be removed from the generated song
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(REST)
special_tokens.remove(WAIT_LSTM)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), WAIT_LSTM, special_tokens))

In [ ]:
new_song = utils.decode_songs_old(generated_song, bpm=120)
print(generated_song)

In [ ]:
path = Path("generated_song_videospielmusik_lstm_mono.mid")
utils.write_midi(new_song, path)

To open the midi file we recomend [musecore](https://musescore.org/de)

## Generate polyphony

In [ ]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [ ]:
from mukkeBude.model import MukkeBudeLSTM
from mukkeBude.mapping import MusicMapping
import mukkeBude.utils as utils
import tensorflow as tf
import numpy as np
from pathlib import Path

from mukkeBude.mapping import SPECIAL_TOKS
from mukkeBude.mapping import SEP
from mukkeBude.mapping import BOS


# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Create mappings
mapping = MusicMapping.create()

# optional save the mapping
# mapping.save("mapping.txt")

In [ ]:
model = MukkeBudeLSTM.load(mapping, "Videospielmusik_polyphonie_lstm")

In [ ]:
# Create song
generated_song = model.generate("n46 d4 xxsep d4 n53 d2 n50 d2 xxsep d2 n62 d1 n58 d1 xxsep d2 n53 d2 n53 d2 n50 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2 n55 d2 n55 d2 n51 d2 xxsep d4 n46 d4 xxsep d4 n57 d2 n53 d2 xxsep d2 n65 d1 n60 d1 xxsep d2 n57 d2 n57 d2 n53 d2 xxsep d4 n46 d4 xxsep d4 n55 d2 n51 d2 xxsep d2 n63 d1 n58 d1 xxsep d2 n55 d2 n55 d2 n51 d2 xxsep d4", 1500, temperature=0.7)

# Remove REST and WAIT_LSTM from SPECIAL_TOKS
# They should not be removed from the generated song
special_tokens = SPECIAL_TOKS.copy()
special_tokens.remove(SEP)
special_tokens.remove(BOS)

generated_song = " ".join(utils.replace_special_tokens(generated_song.split(), "d1", special_tokens))

In [ ]:
new_song_ints = mapping.numericalize(generated_song.split(" "))
new_song = utils.from_polyphonic_encoding(np.array(new_song_ints), mapping, bpm=120)

In [ ]:
path = Path("generated_song_videospielmusik_lstm_poly.mid")
utils.write_midi(new_song, path)